In [53]:
from pyspark.sql.types import *
%load_ext autoreload
%autoreload 2

import pyspark
from delta import *
from dags.lib.IncrementalLoader import IncrementalLoader
from dags.lib.processing_utils import merge_elements
from dags.lib.Processer import *
from pyspark.sql.functions import col, lit, max as spark_max, length, row_number, explode, unix_timestamp, from_unixtime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def create_spark_session():
    conf = (
        pyspark.conf.SparkConf()
        .setAppName("LetsTalk")
        .set(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .set("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
        .set("spark.hadoop.google.cloud.auth.service.account.enable", "true")
        .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/Users/alfio/projects/upc/BDMP2/gcs.json")
        .set("spark.sql.shuffle.partitions", "4")
        .set("spark.jars", "../jars/gcs-connector-hadoop3-latest.jar")
        .setMaster(
            "local[*]"
        )
    )

    builder = pyspark.sql.SparkSession.builder.appName("LetsTalk").config(conf=conf)
    spark = configure_spark_with_delta_pip(builder).getOrCreate()
    return spark



In [5]:
spark = create_spark_session()
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

:: loading settings :: url = jar:file:/Users/alfio/python_venv/general/lib/python3.13/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/alfio/.ivy2/cache
The jars for the packages stored in: /Users/alfio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2d9d07dc-2b0e-4a38-9d37-10ac9f21550a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 67ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0  

In [8]:
absolute_path_to_landing = '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma'
table_subpath = 'delta_tmdb/upcoming'
loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpath)
df = loader.get_new_data()
df

CDF not available — doing full load


adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,ingestion_time,begin_date,end_date
false,/op3qmNhvwEvyT7UF...,"[14, 12, 28]",324544,en,In the Lost Lands,A queen sends the...,727.9871,/iHf6bXPghWB6gT8k...,2025-02-27,In the Lost Lands,false,6.358,180,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/2Nti3gYAX513wvhp...,"[10751, 35, 12, 14]",950387,en,A Minecraft Movie,Four misfits find...,673.4901,/yFHHfHcUgGAxziP1...,2025-03-31,A Minecraft Movie,false,6.1,555,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/3lEV4CoKoeT2cZ4f...,"[27, 9648]",1244944,en,The Woman in the ...,In the aftermath ...,172.9102,/n0WS2TsNcS6dtaZK...,2025-03-27,The Woman in the ...,false,6.27,37,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/wmqpE7p2dUCEgCno...,"[27, 14, 53, 9648...",1353117,en,Home Sweet Home: ...,When a city is ov...,172.3625,/9rCBCm9cyI4JfLEh...,2025-03-20,Home Sweet Home: ...,false,7.096,26,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/6J259Zuq09Bt2PLU...,"[28, 35, 53]",1212855,en,Fight or Flight,A mercenary takes...,99.8608,/x4nWnfgJvL045rcU...,2025-04-03,Fight or Flight,false,5.2,38,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/hwlyY7LJdEFbCPaG...,"[28, 53]",1126166,en,Flight Risk,A U.S. Marshal es...,107.5155,/q0bCG4NX32iIEsRF...,2025-01-22,Flight Risk,false,6.104,629,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/jl2YIADk391yc6Qj...,"[16, 18, 35]",1064486,en,Memoir of a Snail,Forcibly separate...,74.2626,/57AgZv1ITeBLShiN...,2024-10-17,Memoir of a Snail,false,7.948,374,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/xkjWiRQtrenqGRWO...,[27],1388366,en,Popeye the Slayer...,A curious group o...,70.4043,/nVwu3mN7hr1yF467...,2025-03-21,Popeye the Slayer...,false,5.953,75,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/bA4Io3xv46oEyP4O...,"[28, 80]",995926,ko,베테랑 2,The veteran detec...,55.4906,/gAtP0usArK5gVOBO...,2024-09-13,"I, the Executioner",false,6.955,55,2025-04-17 16:59:...,2025-04-23,2025-05-14
false,/9ctmJLf3uUngYkwH...,"[18, 9648, 53]",1233575,en,Black Bag,When intelligence...,45.8759,/hHPovtU4b96LHcoe...,2025-03-12,Black Bag,false,6.3,351,2025-04-17 16:59:...,2025-04-23,2025-05-14


In [5]:
processor = NewsProcessor(spark, df)

In [6]:
processor.ensure_schema()
processor.remove_clear_duplicates()
processor.name_to_id()
processor.remove_hidden_duplicates(['url'], ['publishedAt'])
processor.normalize_text(['title', 'description', 'content'])
processor.expand_source()
processor.order_by('publishedAt', ascending=False)

25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921639145 which is greater than the next commit timestamp 1745921639145.
25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639501 which is greater than the next commit timestamp 1745921639239.
25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639502 which is greater than the next commit timestamp 1745921639185.
25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639603 which is greater than the next commit timestamp 1745921639436.
25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639604 which is greater than the next commit timestamp 1745921639330.
25/05/07 10:36:40 WARN DeltaHistoryManager: Found Delta commit 7 with a timestamp 1745921639605 which is greater than the next commit timestamp 1745921639185.
25/05/07 10:36:40 WARN DeltaHistoryManager: Fo

In [7]:
processor.df

25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921639145 which is greater than the next commit timestamp 1745921639145.
25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921639501 which is greater than the next commit timestamp 1745921639239.
25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921639502 which is greater than the next commit timestamp 1745921639185.
25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921639603 which is greater than the next commit timestamp 1745921639436.
25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921639604 which is greater than the next commit timestamp 1745921639330.
25/05/07 10:36:43 WARN DeltaHistoryManager: Found Delta commit 7 with a timestamp 1745921639605 which is greater than the next commit timestamp 1745921639185.
25/05/07 10:36:43 WARN DeltaHistoryManager: Fo

url,author,content,description,publishedAt,source,title,urlToImage
http://deadline.c...,Natalie Oganesyan,the rock amp ro...,the rock roll h...,2025-04-28 03:33:00,deadline,outkast cyndi la...,https://deadline....
http://www.vultur...,Sara Holdren,the new musical r...,real women have ...,2025-04-28 03:30:23,vulture,bright and bold i...,https://pyxis.nym...
https://deadline....,Glenn Garner,the american film...,stars like elle f...,2025-04-27 04:20:01,deadline,afi life achievem...,https://deadline....
https://www.yourt...,Kate Rose,the love horoscop...,sunday s new moon...,2025-04-27 03:03:23,yourtango,love horoscopes a...,https://www.yourt...
https://www.count...,Anna Logan,did you know gigi...,from hollywood to...,2025-04-26 15:17:46,countryliving.com,celebrities se...,https://hips.hear...
https://www.slash...,Kieran Fisher,the original sno...,how the heck did ...,2025-04-26 15:00:00,/film,the rachel zegler...,https://www.slash...
https://tvline.co...,Matt Webb Mitovich,the following con...,fire country s xl...,2025-04-26 05:19:51,tvline,fire country shoc...,https://tvline.co...
https://tvline.co...,Matt Webb Mitovich,the following con...,ncis sydney sho...,2025-04-26 03:00:00,tvline,ncis sydney boss...,https://tvline.co...
https://www.wwno....,Joseph King,the first weekend...,lil wayne will cl...,2025-04-26 00:30:00,wwno.org,lil wayne s new o...,https://npr.brigh...
http://deadline.c...,Anthony D'Alessandro,there are many qu...,george lucas the ...,2025-04-26 00:08:00,deadline,george lucas on w...,https://deadline....


In [82]:
import os
absolute_path_to_trusted = '/Users/alfio/projects/upc/BDMP2/data/letstalk_trusted_zone_bdma'
save_path = os.path.join(absolute_path_to_trusted, table_subpath)

processor.df = processor.df.withColumn(
    "url",
    when(col('author') == "Sara Holdren", "https:test.com").otherwise(col("url"))
)

In [84]:
processor.merge_with_trusted(save_path, ['url'])

INFO:root:Saving unique records from overlapping ones
INFO:root:Added new 1 unique records
INFO:root:Appending non overlapping records
INFO:root:Adding new 0 records


In [4]:
absolute_path_to_landing = '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma'
table_subpath = 'delta_sports/leagues'
loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpath)
df = loader.get_new_data()
df.head(5)

25/05/17 17:51:01 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/17 17:51:05 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921641125 which is greater than the next commit timestamp 1745921641094.
25/05/17 17:51:05 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921641126 which is greater than the next commit timestamp 1745921641094.
25/05/17 17:51:05 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921641127 which is greater than the next commit timestamp 1745921641094.
25/05/17 17:51:05 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921641128 which is greater than the next commit timestamp 1745921641064.
25/05/17 17:51:05 WARN DeltaHistoryManager: Found Delta commit 6 with a timestamp 1745921641244 which is greater than the next commit timestamp 1745921641214.
25/05/17 17:51:0

CDF not available — doing full load


[Row(country=Row(code=None, flag=None, name='World'), league=Row(id=4, logo='https://media.api-sports.io/football/leagues/4.png', name='Euro Championship', type='Cup'), seasons=[Row(coverage=Row(fixtures=Row(events=True, lineups=True, statistics_fixtures=False, statistics_players=False), injuries=False, odds=False, players=True, predictions=True, standings=False, top_assists=True, top_cards=True, top_scorers=True), current=False, end='2008-06-29', start='2008-06-07', year=2008), Row(coverage=Row(fixtures=Row(events=True, lineups=True, statistics_fixtures=False, statistics_players=False), injuries=False, odds=False, players=True, predictions=True, standings=False, top_assists=True, top_cards=True, top_scorers=True), current=False, end='2012-07-01', start='2012-06-08', year=2012), Row(coverage=Row(fixtures=Row(events=True, lineups=True, statistics_fixtures=True, statistics_players=True), injuries=False, odds=False, players=True, predictions=True, standings=True, top_assists=True, top_car

In [5]:
sporcessor = SportsLeagueProcessor(spark, df)
countries = sporcessor.generate_countries()
leagues = sporcessor.generate_leagues()
sporcessor.expand()

 Changing code, but probably a manual check could be needed


In [6]:
sporcessor.df

league_id,league-info_current,league-info_end,league-info_start,league-info_year,coverage_injuries,coverage_odds,coverage_players,coverage_predictions,coverage_standings,coverage_top_assists,coverage_top_cards,coverage_top_scorers,fixture_events,fixture_lineups,fixture_statistics_fixtures,fixture_statistics_players
4,false,2008-06-29,2008-06-07,2008,false,false,true,true,false,true,true,true,true,true,false,false
4,false,2012-07-01,2012-06-08,2012,false,false,true,true,false,true,true,true,true,true,false,false
4,false,2016-07-10,2016-06-10,2016,false,false,true,true,true,true,true,true,true,true,true,true
4,false,2021-07-11,2019-03-21,2020,false,false,true,true,true,true,true,true,true,true,true,true
4,true,2024-07-14,2024-06-14,2024,false,false,true,true,true,true,true,true,true,true,true,true
21,false,2009-06-28,2009-06-14,2009,false,false,true,true,false,true,true,true,true,true,false,false
21,false,2013-06-30,2013-06-15,2013,false,false,true,true,false,true,true,true,true,true,false,false
21,true,2017-07-02,2017-06-17,2017,false,false,true,true,false,true,true,true,true,true,true,false
61,false,2011-05-29,2010-08-07,2010,false,false,true,true,true,true,true,true,true,true,false,false
61,false,2012-05-20,2011-08-06,2011,false,false,true,true,true,true,true,true,true,true,false,false


In [23]:
absolute_path_to_landing = '/Users/alfio/projects/upc/BDMP2/data/letstalk_landing_zone_bdma'
table_subpath = 'delta_sports/matches'
loader = IncrementalLoader(spark, absolute_path_to_landing, table_subpath)
df = loader.get_new_data()
df.head(5)

25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 0 with a timestamp 1745921644516 which is greater than the next commit timestamp 1745921644042.
25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 1 with a timestamp 1745921644517 which is greater than the next commit timestamp 1745921643917.
25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 2 with a timestamp 1745921644518 which is greater than the next commit timestamp 1745921642587.
25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 3 with a timestamp 1745921644519 which is greater than the next commit timestamp 1745921644158.
25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 4 with a timestamp 1745921644520 which is greater than the next commit timestamp 1745921642587.
25/05/17 19:09:05 WARN DeltaHistoryManager: Found Delta commit 5 with a timestamp 1745921644521 which is greater than the next commit timestamp 1745921642590.
25/05/17 19:09:05 WARN DeltaHistoryManager: Fo

CDF not available — doing full load


[Row(fixture=Row(date='2025-04-25T00:00:00+00:00', id=1345365, periods=Row(first=1745539200, second=1745542800), referee='R. Homik', status=Row(elapsed=90, extra=7, long='Match Finished', short='FT'), timestamp=1745539200, timezone='UTC', venue=Row(city='Commerce City, Colorado', id=None, name='Dick&amp;apos;s Sporting Goods Park')), goals=Row(away=2, home=0), league=Row(country='USA', flag='https://media.api-sports.io/flags/us.svg', id=909, logo='https://media.api-sports.io/football/leagues/909.png', name='MLS Next Pro', round='Regular Season - 8', season=2025, standings=True), score=Row(extratime=Row(away=None, home=None), fulltime=Row(away=2, home=0), halftime=Row(away=2, home=0), penalty=Row(away=None, home=None)), teams=Row(away=Row(id=18814, logo='https://media.api-sports.io/football/teams/18814.png', name='Minnesota United II', winner=True), home=Row(id=4102, logo='https://media.api-sports.io/football/teams/4102.png', name='Colorado Rapids II', winner=False))),
 Row(fixture=Row(

In [51]:
sproc = SportsMatchesProcessor(spark, df)
sproc.expand()
sproc.normalize_text(['referee', 'venue_city', 'venue_name', 'team_away_name', 'team_home_name', 'status_long'])
sproc.extract_teams()
sproc.extract_venues()
sproc.remove_useless_columns()
sproc.ensure_schema()

In [52]:
sproc.df

league,fixture_date,fixture_id,period_first,period_second,referee,status_elapsed,status_extra,status_long,timestamp,venue_id,team_away_id,team_home_id,goals_away,goals_home
909,2025-04-25T00:00:...,1345365,2025-04-25T02:00:...,2025-04-25T03:00:...,r homik,90,7,match finished,2025-04-25T02:00:...,dick_amp_apos_s_s...,18814,4102,2,0
243,2025-04-25T00:00:...,1354612,2025-04-25T02:00:...,2025-04-25T03:00:...,NULL,90,5,match finished,2025-04-25T02:00:...,estadio_monumenta...,1159,10112,1,1
13,2025-04-25T00:00:...,1361899,2025-04-25T02:00:...,2025-04-25T03:00:...,mario alberto dia...,90,5,match finished,2025-04-25T02:00:...,casa_de_apostas_a...,1137,118,0,1
263,2025-04-25T00:00:...,1372443,2025-04-25T02:00:...,2025-04-25T03:00:...,m l pez s nchez,90,8,match finished,2025-04-25T02:00:...,estadio_ol_mpico_...,2299,2311,0,2
13,2025-04-25T00:30:...,1361901,2025-04-25T02:30:...,2025-04-25T03:30:...,esteban ostojich ...,90,5,match finished,2025-04-25T02:30:...,estadio_ueno_la_n...,2546,1176,2,2
11,2025-04-25T00:30:...,1362132,2025-04-25T02:30:...,2025-04-25T03:30:...,mathias de armas ...,90,NULL,match finished,2025-04-25T02:30:...,estadio_bicentena...,135,2318,1,2
11,2025-04-25T00:30:...,1362139,2025-04-25T02:30:...,2025-04-25T03:30:...,nicolas ramirez ...,90,12,match finished,2025-04-25T02:30:...,estadio_tierra_de...,2562,2319,2,2
72,2025-04-25T00:35:...,1353308,2025-04-25T02:35:...,2025-04-25T03:35:...,denis da silva ri...,90,5,match finished,2025-04-25T02:35:...,est_dio_dr__jorge...,134,7834,1,2
422,2025-04-25T01:00:...,1337904,NULL,NULL,NULL,NULL,NULL,not started,2025-04-25T03:00:...,valerie_playing_f...,8406,8417,NULL,NULL
673,2025-04-25T01:00:...,1372193,2025-04-25T03:00:...,2025-04-25T04:00:...,NULL,90,8,match finished,2025-04-25T03:00:...,estadio_akron,14870,14876,0,2


In [40]:
print(df_ex.filter(col('team_home_winner') == True).filter(col('goals_away') > col('goals_home')).count())
print(df_ex.filter(col('team_away_winner') == True).filter(col('goals_home') > col('goals_away')).count())

0
0


In [29]:
df_ex.filter(col('teams.away.name').startswith('Juventus')).head()

Row(goals=Row(away=1, home=3), league=139, teams=Row(away=Row(id=1886, logo='https://media.api-sports.io/football/teams/1886.png', name='Juventus W', winner=False), home=Row(id=1880, logo='https://media.api-sports.io/football/teams/1880.png', name='Fiorentina W', winner=True)), fixture_date='2025-04-25T10:30:00+00:00', fixture_id=1351012, period_first=1745577000, period_second=1745580600, referee=None, status_elapsed=90, status_extra=5, status_long='Match Finished', status_short='FT', timestamp=1745577000, timezone='UTC', venue_city='Bagno a Ripoli', venue_id=21168, venue_name='Stadio Curva Fiesole - Viola Park')